https://labs.vocareum.com/main/main.php?m=editor&nav=1&asnid=1192746&stepid=1192747

# hmmlearn.py

In [2]:
import numpy as np 
import pandas as pd
import copy
import re
import sys

filename = sys.argv[1]
#fp = open("hmm-training-data/it_isdt_train_tagged.txt","r", encoding='utf8')
fp = open(filename,"r", encoding='utf8')
lines = fp.readlines()
transitions = {}
emisions = {}

i = 0
for line in lines:
    line_list = line.split()
    for word in line_list:
        pos = word.split('/')[-1]
        n = len(word) - len(pos) - 1
        emision = word[:n]
        
        if(pos in transitions.keys()):
            transitions[pos] += 1
        else:
            transitions[pos] = 1
        
        if(emision in emisions.keys()):
            emisions[emision] += 1
        else:
            emisions[emision] = 1

#print("The tags are: ", transitions, "Count: ", len(transitions))
#print("The words are: ", emisions, "Count: ", len(emisions))

In [3]:
emision_matrix = {}
transition_matrix = {}

for line in lines:
    line_list = line.split()
    prev_tag = ""
    for i in range(len(line_list)):
        word = line_list[i]
        pos = word.split('/')[-1]
        n = len(word) - len(pos) - 1
        emision = word[:n]

        if(emision in emision_matrix.keys()):
            if(pos in emision_matrix[emision].keys()):
                emision_matrix[emision][pos] += 1
            else:
                emision_matrix[emision][pos] = 1
        else:
            emision_matrix[emision] = {pos: 1}

        if(i==0):
            prev_tag = pos
            continue

        if(prev_tag in transition_matrix.keys()):
            if(pos in transition_matrix[prev_tag].keys()):
                transition_matrix[prev_tag][pos] += 1
            else:
                transition_matrix[prev_tag][pos] = 1
        else:
            transition_matrix[prev_tag] = {pos: 1}
        prev_tag = pos
#transition_matrix
#emision_matrix

In [4]:
for i in emision_matrix.keys():
    total = sum(emision_matrix[i].values())
    emision_matrix[i] = {key: value/total for key, value in emision_matrix[i].items()}

for i in transition_matrix.keys():
    total = sum(transition_matrix[i].values())
    transition_matrix[i] = {key: value/total for key, value in transition_matrix[i].items()}

#transition_matrix = {key: value / total for key, value in transition_matrix.items()}
print(emision_matrix.keys())
print(transition_matrix.keys())
#print(transition_matrix)

dict_keys(['LONDRA', '.', 'Gas', 'da', 'la', 'statua', 'Inconsueto', 'allarme', 'a', 'Tate', 'Gallery', ':', 'una', 'sala', 'ha', 'dovuto', 'essere', 'sgomberata', 'per', 'fuga', 'di', 'gas', 'tossico', 'scultura', 'moderna', 'in', 'vetro', 'che', 'simboleggia', '"', 'i', 'pericoli', 'vita', 'Rilevata', 'presenza', 'uno', 'tubi', 'trasparenti', 'compongono', "l'", 'opera', ',', 'guardiani', 'hanno', 'fatto', 'scattare', 'speciale', 'piano', "d'", 'emergenza', 'e', '45', 'minuti', 'pompieri', 'isolato', 'La', 'è', 'il', 'pachistano', 'Hamad', 'Butt', 'intitolata', 'Riti', 'Passaggio', 'forma', 'scala', 'contiene', 'iodio', 'scaldato', 'le', 'luci', 'si', 'trasformato', 'morto', '1994', '32', 'anni', 'Numerose', 'sue', 'opere', 'contengono', 'prodotti', 'chimici', 'tossici', 'Trionfo', 'Didoni', '20', 'chilometri', 'marcia', 'Fiona', 'May', 'lungo', 'Doppio', 'oro', 'azzurri', 'mondiali', 'atletica', 'cominciato', 'Michele', 'milanese', 'Quarto', 'Oggiaro', 'con', 'titolo', 'km', '(', 'D

In [5]:
a = {"x":{"s":2,"t":4},"y":{"s":3,"t":6},"z":{"s":4,"t":8}}
a["x"].keys()

dict_keys(['s', 't'])

# hmmcode.py

In [32]:
def tag_a_line(line):
    alpha = 0.000001
    output_matrix = []
    blank = {k: [0,""] for k in transition_matrix.keys()}
    flag_first = 1
    prev_word = ""

    for i in range(len(line)):
        output_matrix.append(copy.deepcopy(blank))
        if flag_first == 1:
            try:
                for e_key in emision_matrix[line[i]].keys():
                    output_matrix[i][e_key][0] = emision_matrix[line[i]][e_key]
                    output_matrix[i][e_key][1] = e_key
                flag_first = 0
                prev_word = i
                continue
            except:
                for e_key in output_matrix[i].keys():
                    output_matrix[i][e_key][0] = alpha
                    output_matrix[i][e_key][1] = e_key
                flag_first = 0
                prev_word = i
                continue

        tag = ''
        result = 0

        for pos in output_matrix[i].keys():
            tag = ''
            result = 0
            for pos_prev in output_matrix[prev_word].keys():
                prev_result = result
                prior = output_matrix[prev_word][pos_prev][0] if(output_matrix[prev_word][pos_prev][0] != 0) else alpha
                try:
                    t = transition_matrix[pos_prev][pos]
                except:
                    t = alpha
                try:
                    e = emision_matrix[line[i]][pos]
                except:
                    e = alpha
                result = max(result, prior * t * e)
                tag =  pos_prev if (result != prev_result) else tag

            output_matrix[i][pos] = [result,tag]

    #output_matrix.keys()
    solution = []
    word = line[-1]
    x = max(output_matrix[-1], key=output_matrix[-1].get)
    solution.append(x)
    solution.append(output_matrix[-1][x][1])
    for i in range(len(line)-3,-1,-1):
        solution.append(output_matrix[i][solution[-1]][1])
    return solution


In [37]:
fp = open("hmm-training-data/it_isdt_dev_raw.txt","r", encoding='utf8')
lines = fp.readlines()
for line in lines:
    x = line.strip().split()
    s = tag_a_line(x)
    with open('output.txt', 'a', encoding='utf8') as f:
        for i in range(0,len(x)):
            f.write(x[i])
            f.write("/")
            f.write(s[i])
            f.write(" ")
        f.write("\n")
f.close()